# Utility functions

In [1]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, balanced_accuracy_score,  classification_report, ConfusionMatrixDisplay

def show_ConfusionMatrix_test(y_test, y_test_pred, test_confusion_matrix_title = "Confusion Matrix (Test)"):
  conf_matrix_log_reg = confusion_matrix(y_test, y_test_pred)
  print("Confusion Matrix (Test):")
  print(conf_matrix_log_reg)
  ConfusionMatrixDisplay(conf_matrix_log_reg).plot()
  plt.title(test_confusion_matrix_title)
  plt.show()
'''
def find_optimal_threshold(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters:
    target : Matrix with dependent or target data, where rows are observations
    predicted : Matrix with predicted data, where rows are observations

    Returns:
    list type, with optimal cutoff value
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr))
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]
    return list(roc_t['threshold'])
'''

def get_precision_and_recall(y, y_pred):
  return round(precision_score(y, y_pred), 4), round(recall_score(y, y_pred), 4)

# Download dataset from Kaggle

In [2]:
# Install Kaggle API
# !pip install kaggle

# # Make a directory for the Kaggle credentials and move the JSON file there
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# # Set permissions for the JSON file
# !chmod 600 ~/.kaggle/kaggle.json

# !cd ../data/

# # Download the dataset
# !kaggle datasets download -d manishkc06/startup-success-prediction

# # Unzip the downloaded file
# !unzip startup-success-prediction.zip

# !ls
%pip install yfinance
%pip install tweepy
%pip install wbdata pandas



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Explore and adjust data

## functions

### add economic info

In [16]:
# Affect acquisitions
# Define the function to add NASDAQ annual changes
# def add_nasdaq_annual_changes(df):
#     years = np.arange(1984, 2024).tolist()
#     nasdaq_data = yf.download('^IXIC', start='1984-01-01', end='2024-01-01', interval='1mo')
#     nasdaq_data['Year'] = nasdaq_data.index.year
#     nasdaq_annual = nasdaq_data.groupby('Year')['Close'].last().pct_change().reset_index()
#     nasdaq_annual.columns = ['Year', 'NASDAQ_Annual_Change']
#     nasdaq_annual = nasdaq_annual[nasdaq_annual['Year'].isin(years)]
#     nasdaq_annual.set_index('Year', inplace=True)

#     for i in range(11):
#         df[f'nasdaq_annual_changes_at_year_{i}'] = df['founded_at_year'].apply(lambda x: nasdaq_annual['NASDAQ_Annual_Change'].get(x-i, np.nan))

#     return df


def add_nasdaq_annual_changes(df):
    years = np.arange(1984, 2024).tolist()
    nasdaq_data = yf.download('^IXIC', start='1984-01-01', end='2024-01-01', interval='1mo')
    nasdaq_data['Year'] = nasdaq_data.index.year
    nasdaq_annual = nasdaq_data.groupby('Year')['Close'].last().pct_change().reset_index()
    nasdaq_annual.columns = ['Year', 'NASDAQ_Annual_Change']
    nasdaq_annual = nasdaq_annual[nasdaq_annual['Year'].isin(years)]
    nasdaq_annual.set_index('Year', inplace=True)
    
    # Add columns for each year based on the maximum founded_at_year in the DataFrame
    max_year = df['founded_at_year'].max()
    for year in range(max_year, max_year - 11, -1):
        year_index = max_year - year
        df[f'nasdaq_annual_changes_at_year_{year_index}'] = df['founded_at_year'].apply(
            lambda x: nasdaq_annual.loc[year, 'NASDAQ_Annual_Change'] if year in nasdaq_annual.index else np.nan
        )
    
    return df


# Affect workforce and investments
def add_info_on_inflation_over_years(df):
  us_inflation_data = {
      1984: 4.3, 1985: 3.6, 1986: 1.9, 1987: 3.6, 1988: 4.1, 1989: 4.8,
      1990: 5.4, 1991: 4.2, 1992: 3.0, 1993: 2.7, 1994: 2.6, 1995: 2.8,
      1996: 3.0, 1997: 2.3, 1998: 1.6, 1999: 2.2, 2000: 3.4, 2001: 2.8,
      2002: 1.6, 2003: 2.3, 2004: 2.7, 2005: 3.4, 2006: 3.2, 2007: 2.8,
      2008: 3.8, 2009: -0.4, 2010: 1.6, 2011: 3.2, 2012: 2.1, 2013: 1.5,
      2014: 1.6, 2015: 0.1, 2016: 1.3, 2017: 2.1, 2018: 2.4, 2019: 1.8,
      2020: 1.2, 2021: 4.7, 2022: 8.0, 2023: 3.2, 2024: 2.5  # 2024 is an estimate
  }

  df['inflation_at_year_0'] = df['founded_at_year'].map(us_inflation_data)
  df['inflation_at_year_1'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-1, None))
  df['inflation_at_year_2'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-2, None))
  df['inflation_at_year_3'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-3, None))

  df['inflation_at_year_4'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-4, None))
  df['inflation_at_year_5'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-5, None))
  df['inflation_at_year_6'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-6, None))
  df['inflation_at_year_7'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-7, None))
  df['inflation_at_year_8'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-8, None))
  df['inflation_at_year_9'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-9, None))
  df['inflation_at_year_10'] = df.founded_at_year.apply(lambda x: us_inflation_data.get(x-10, None))

  return df

# Affect workforce and ivestments
def add_us_gdp_growth_data_over_years(df):
    us_gdp_growth_data = {
        1984: 7.3, 1985: 4.2, 1986: 3.5, 1987: 3.5, 1988: 4.2, 1989: 3.7,
        1990: 1.9, 1991: -0.1, 1992: 3.6, 1993: 2.8, 1994: 4.0, 1995: 2.7,
        1996: 3.8, 1997: 4.5, 1998: 4.5, 1999: 4.8, 2000: 4.1, 2001: 1.0,
        2002: 1.7, 2003: 2.9, 2004: 3.8, 2005: 3.5, 2006: 2.9, 2007: 1.9,
        2008: -0.1, 2009: -2.5, 2010: 2.6, 2011: 1.6, 2012: 2.2, 2013: 1.8,
        2014: 2.5, 2015: 3.1, 2016: 1.6, 2017: 2.4, 2018: 2.9, 2019: 2.3,
        2020: -3.4, 2021: 5.9, 2022: 2.1, 2023: 2.0, 2024: 2.3  # 2024 is an estimate
    }

    df['gdp_growth_at_year_0'] = df['founded_at_year'].map(us_gdp_growth_data)
    df['gdp_growth_at_year_1'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-1, None))
    df['gdp_growth_at_year_2'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-2, None))
    df['gdp_growth_at_year_3'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-3, None))

    df['gdp_growth_at_year_4'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-4, None))
    df['gdp_growth_at_year_5'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-5, None))
    df['gdp_growth_at_year_6'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-6, None))
    df['gdp_growth_at_year_7'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-7, None))
    df['gdp_growth_at_year_8'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-8, None))
    df['gdp_growth_at_year_9'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-9, None))
    df['gdp_growth_at_year_10'] = df.founded_at_year.apply(lambda x: us_gdp_growth_data.get(x-10, None))

    return df

# Affect workforce
def add_us_workforce_growth_data_over_years(df):
    us_workforce_growth_data = {
        1984: 1.5, 1985: 1.6, 1986: 1.5, 1987: 1.7, 1988: 1.6, 1989: 1.4,
        1990: 1.3, 1991: 1.1, 1992: 1.3, 1993: 1.2, 1994: 1.4, 1995: 1.2,
        1996: 1.3, 1997: 1.2, 1998: 1.3, 1999: 1.2, 2000: 1.2, 2001: 0.9,
        2002: 0.8, 2003: 0.7, 2004: 1.1, 2005: 1.2, 2006: 1.1, 2007: 1.1,
        2008: 0.7, 2009: -0.3, 2010: 0.7, 2011: 0.7, 2012: 1.0, 2013: 0.9,
        2014: 1.0, 2015: 1.2, 2016: 1.1, 2017: 1.0, 2018: 1.3, 2019: 1.0,
        2020: -2.3, 2021: 2.5, 2022: 1.8, 2023: 1.0, 2024: 1.1  # 2024 is an estimate
    }

    df['workforce_growth_at_year_0'] = df['founded_at_year'].map(us_workforce_growth_data)
    df['workforce_growth_at_year_1'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-1, None))
    df['workforce_growth_at_year_2'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-2, None))
    df['workforce_growth_at_year_3'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-3, None))

    df['workforce_growth_at_year_4'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-4, None))
    df['workforce_growth_at_year_5'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-5, None))
    df['workforce_growth_at_year_6'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-6, None))
    df['workforce_growth_at_year_7'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-7, None))
    df['workforce_growth_at_year_8'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-8, None))
    df['workforce_growth_at_year_9'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-9, None))
    df['workforce_growth_at_year_10'] = df.founded_at_year.apply(lambda x: us_workforce_growth_data.get(x-10, None))

    return df


def add_economic_indicators(df, indicator_code):
    # Define the fixed range of years
    years_range = 11
    start_year = df['founded_at_year'].min() - years_range
    end_year = df['founded_at_year'].max()

    # Fetch data from World Bank for a broad range of years
    countries = 'US'
    data = wbdata.get_dataframe({indicator_code: 'Economic_Indicator'}, country=countries)
    
    # Reset index and rename columns
    data = data.reset_index()
    data.rename(columns={indicator_code: 'Economic_Indicator'}, inplace=True)
    
    # Ensure 'date' column is in datetime format
    data['date'] = pd.to_datetime(data['date'], errors='coerce')
    
    # Extract year from the date and filter data
    data['Year'] = data['date'].dt.year
    data = data.set_index(['Year'])
    
    # Filter data to include only the years of interest
    years_of_interest = np.arange(start_year, end_year + 1)
    data = data.loc[data.index.isin(years_of_interest)]
    
    # Add columns for each year in the fixed range
    for i in range(years_range):
        if 'GDP' in indicator_code:
            key_word = 'GDP'
        elif 'UEM' in indicator_code:
            key_word = 'UEM'
        else:
            key_word = 'UNKNOWN'

        year = end_year - i
        df[f'{key_word}_at_year_{i}'] = df['founded_at_year'].apply(
            lambda x: data.loc[year, 'Economic_Indicator'] if year in data.index else np.nan
        )
    return df



### add & adjust data

In [15]:
def read_and_adjust(file_name = 'startup data.csv'):
  # Load the data
  df = pd.read_csv(file_name)
  
  # Create lable
  df['status_code'] = df['status'].map({'acquired': 1, 'closed': 0})

  # Convert the date column to datetime
  df['founded_at_date'] = pd.to_datetime(df['founded_at'])

  # Extract year, month, and day into separate columns
  df['founded_at_year'] = df['founded_at_date'].dt.year
  df['founded_at_month'] = df['founded_at_date'].dt.month
  df['founded_at_day'] = df['founded_at_date'].dt.day

  # Drop colums
  df = df.drop(columns=['status', 'founded_at', 'name', 'id', 'state_code', 'object_id', 'category_code', 'labels', 'closed_at'])

  #df = add_info_on_inflation_over_years(df)
  # df = add_us_gdp_growth_data_over_years(df)
  #df = add_us_workforce_growth_data_over_years(df)
  
  df = add_nasdaq_annual_changes(df)
  
  indicator_code = 'NY.GDP.MKTP.KD.ZG'
  df = add_economic_indicators(df, indicator_code) # will replace add_us_gdp_growth_data_over_years
  
  indicator_code = 'SL.UEM.TOTL.ZS'  # Unemployment rate, percentage of total labor force
  df = add_economic_indicators(df, indicator_code) # will replace add_us_gdp_growth_data_over_years


  return df


## run

In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
import datetime
# import tweepy
import wbdata

dataset_path = '../data/startup_data.csv'
df = read_and_adjust(file_name = dataset_path)

# Display the first few rows of the data
# df.head()


[*********************100%%**********************]  1 of 1 completed


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 78 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Unnamed: 0                        923 non-null    int64         
 1   latitude                          923 non-null    float64       
 2   longitude                         923 non-null    float64       
 3   zip_code                          923 non-null    object        
 4   city                              923 non-null    object        
 5   Unnamed: 6                        430 non-null    object        
 6   first_funding_at                  923 non-null    object        
 7   last_funding_at                   923 non-null    object        
 8   age_first_funding_year            923 non-null    float64       
 9   age_last_funding_year             923 non-null    float64       
 10  age_first_milestone_year          771 non-null    

In [ ]:
df.founded_at_year.value_counts()

In [ ]:
df.info()

In [ ]:
df.columns

# PCA

### Number of components needed

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

thereshold_PCA = 0.85  # 3 components will explain only 20% of the data variance

#df = df.drop('status')
#df = df.drop('status_code')

# Identify categorical and numerical columns
categorical_columns = df.select_dtypes(include=['object']).columns
numerical_columns = df.select_dtypes(include=['number']).columns

# OneHotEncode categorical columns
encoder = OneHotEncoder()
encoded_categorical = encoder.fit_transform(df[categorical_columns]).toarray()

# Create a DataFrame from the encoded categorical data
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))

# Standardize numerical columns
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df[numerical_columns])

# Create a DataFrame from the scaled numerical data
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_columns)

# Combine encoded categorical and scaled numerical data
processed_df = pd.concat([encoded_categorical_df, scaled_numerical_df], axis=1)

# Replace NaN values with the average value of their respective columns
processed_df.fillna(processed_df.mean(), inplace=True)

# Perform PCA
pca = PCA()
principalComponents = pca.fit_transform(processed_df)

# Explained variance
explained_variance = pca.explained_variance_ratio_

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance)

# Find the number of components that explain at least thereshold_PCA of the variance
num_components = np.argmax(cumulative_explained_variance >= thereshold_PCA) + 1

print(f'Minimal number of components to explain {thereshold_PCA} variance: {num_components}')

# Plot the cumulative explained variance
plt.figure(figsize=(10, 6))
plt.plot(cumulative_explained_variance, marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.axhline(y=thereshold_PCA, color='r', linestyle='-')
plt.axvline(x=num_components-1, color='r', linestyle='-')
plt.grid()
plt.show()

### PCA with 3 components

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Separate the 'status' column
status = df['status_code']

# Exclude the 'status' column from the features
df_pca = df.drop(columns=['status_code'])

# Identify categorical and numerical columns
categorical_columns = df_pca.select_dtypes(include=['object']).columns
numerical_columns = df_pca.select_dtypes(include=['number']).columns

# OneHotEncode categorical columns
encoder = OneHotEncoder()
encoded_categorical = encoder.fit_transform(df_pca[categorical_columns]).toarray()

# Create a DataFrame from the encoded categorical data
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))

# Standardize numerical columns
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df_pca[numerical_columns])

# Create a DataFrame from the scaled numerical data
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_columns)

# Combine encoded categorical and scaled numerical data
processed_df = pd.concat([encoded_categorical_df, scaled_numerical_df], axis=1)

# Replace NaN values with the average value of their respective columns
processed_df.fillna(processed_df.mean(), inplace=True)

# Perform PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(processed_df)

# Create a DataFrame with the PCA results and the status
pca_df = pd.DataFrame(data=principalComponents, columns=['component_1', 'component_2', 'component_3'])
pca_df['status'] = status.values

# Plot the first three PCA components in 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
for target in pca_df['status'].unique():
    subset = pca_df[pca_df['status'] == target]
    ax.scatter(subset['component_1'], subset['component_2'], subset['component_3'], label=target, alpha=0.5)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.set_title('PCA Visualization (First 3 Components)')
ax.legend()
plt.show()

# TSN-E

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

df_tsne = df

# Identify categorical and numerical columns
categorical_columns = df_tsne.select_dtypes(include=['object']).columns
numerical_columns = df_tsne.select_dtypes(include=['number']).columns

# OneHotEncode categorical columns
encoder = OneHotEncoder()
encoded_categorical = encoder.fit_transform(df_tsne[categorical_columns]).toarray()

# Standardize numerical columns
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df_tsne[numerical_columns])

# Combine encoded categorical and scaled numerical data
processed_data = np.hstack((encoded_categorical, scaled_numerical))

# Create a DataFrame with the processed data
processed_df = pd.DataFrame(processed_data)

# Replace NaN values with the average value of their respective columns
processed_df.fillna(processed_df.mean(), inplace=True)


# Perform t-SNE
tsne = TSNE(n_components=3, random_state=42)
tsne_components = tsne.fit_transform(processed_df)

# Create a DataFrame with the t-SNE results and the status
tsne_df = pd.DataFrame(data=tsne_components, columns=['component_1', 'component_2', 'component_3'])
tsne_df['status'] = status.values

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
for target in tsne_df['status'].unique():
    subset = tsne_df[tsne_df['status'] == target]
    ax.scatter(subset['component_1'], subset['component_2'], subset['component_3'], label=target, alpha=0.5)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.set_title('t-SNE  Visualization (First 3 Components)')
ax.legend()
plt.show()

# Data

In [ ]:
df.columns

In [ ]:
df[['founded_at', 'first_funding_at', 'last_funding_at', 'state_code.1', 'has_roundC', 'has_roundD']].head()

# XGBoost classifier

## train model

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt

df = read_and_adjust(file_name = 'startup data.csv')

# Separate the 'status_code' column (target) and features
X = df.drop(columns=['status_code'])
y = df['status_code']

# Identify categorical and numerical columns
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# OneHotEncode categorical columns
encoder = OneHotEncoder(sparse=False)
encoded_categorical = encoder.fit_transform(X[categorical_columns])

# Create DataFrame from encoded categorical data
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))

# Standardize numerical columns
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(X[numerical_columns])

# Create DataFrame from scaled numerical data
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_columns)

# Combine encoded categorical and scaled numerical data
processed_df = pd.concat([encoded_categorical_df, scaled_numerical_df], axis=1)

# Replace NaN values with the average value of their respective columns
processed_df.fillna(processed_df.mean(), inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(processed_df, y, test_size=0.2, random_state=20)

# Train an XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(X_train, y_train)

# Make predictions and predict probabilities
y_pred = xgb_clf.predict(X_test)
y_prob = xgb_clf.predict_proba(X_test)[:, 1]

th = 0.7 #find_optimal_threshold(y_test, y_pred)

print(f'optimal threshold is {th}')
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred > th)
report = classification_report(y_test, y_pred > th)
auc_roc = roc_auc_score(y_test, y_prob)
print(f'AUC-ROC: {round(auc_roc, 2)}')
print(f'Accuracy: {round(accuracy, 2)}')
print(f'Ballanced Accuracy: {round(balanced_accuracy_score(y_test, y_pred > th), 2)}')
print('Classification Report:')
print(report)
show_ConfusionMatrix_test(y_test, y_pred > th)
precision, recall = get_precision_and_recall(y_test,  y_pred > th)
print(f"Precision (Test): {precision}")
print(f"Recall (Test): {recall}")

## feature importance

In [ ]:
# Plot top 10 feature importances
importance = xgb_clf.feature_importances_
feature_names = processed_df.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False).head(15)

plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.title('Top 10 Feature Importances')
plt.gca().invert_yaxis()
plt.show()

## AUC-ROC curve

In [ ]:
# Plot the AUC-ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC-ROC (area = {auc_roc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

## cross - validation

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, make_scorer, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = read_and_adjust(file_name = 'startup data.csv')

# Separate the 'status_code' column (target) and features
X = df.drop(columns=['status_code'])
y = df['status_code']

# Identify categorical and numerical columns
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# OneHotEncode categorical columns
encoder = OneHotEncoder(sparse=False)
encoded_categorical = encoder.fit_transform(X[categorical_columns])

# Create DataFrame from encoded categorical data
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))

# Standardize numerical columns
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(X[numerical_columns])

# Create DataFrame from scaled numerical data
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_columns)

# Combine encoded categorical and scaled numerical data
processed_df = pd.concat([encoded_categorical_df, scaled_numerical_df], axis=1)

# Replace NaN values with the average value of their respective columns
processed_df.fillna(processed_df.mean(), inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(processed_df, y, test_size=0.2, random_state=20)

# Train an XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Perform 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'accuracy': 'accuracy',
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'roc_auc': 'roc_auc',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'kappa': make_scorer(cohen_kappa_score)
}

cv_results = cross_validate(xgb_clf, X_train, y_train, cv=cv, scoring=scoring, return_train_score=False)
print(f"Cross-Validation Accuracy Scores: {cv_results['test_accuracy']}")
print(f"Mean Cross-Validation Accuracy: {round(cv_results['test_accuracy'].mean(), 2)}")
print(f"Cross-Validation Balanced Accuracy Scores: {cv_results['test_balanced_accuracy']}")
print(f"Mean Cross-Validation Balanced Accuracy: {round(cv_results['test_balanced_accuracy'].mean(), 2)}")
print(f"Cross-Validation AUC Scores: {cv_results['test_roc_auc']}")
print(f"Mean Cross-Validation AUC: {round(cv_results['test_roc_auc'].mean(), 2)}")
print(f"Cross-Validation Precision Scores: {cv_results['test_precision']}")
print(f"Mean Cross-Validation Precision: {round(cv_results['test_precision'].mean(), 2)}")
print(f"Cross-Validation Recall Scores: {cv_results['test_recall']}")
print(f"Mean Cross-Validation Recall: {round(cv_results['test_recall'].mean(), 2)}")
print(f"Cross-Validation F1 Scores: {cv_results['test_f1']}")
print(f"Mean Cross-Validation F1: {round(cv_results['test_f1'].mean(), 2)}")
print(f"Cross-Validation Kappa Scores: {cv_results['test_kappa']}")
print(f"Mean Cross-Validation Kappa: {round(cv_results['test_kappa'].mean(), 2)}")

# Statistics

In [ ]:
from scipy.stats import spearmanr, pearsonr

status_code = df['status_code']

def print_correlations_Spearman_and_Pearson(col1, status_code, text = ''):
  # Calculate Spearman and Pearson correlations
  spearman_corr, spearman_p_value = spearmanr(col1, status_code)
  pearson_corr, pearson_p_value = pearsonr(col1, status_code)

  print(f"Spearman correlation: {spearman_corr}, p-value: {spearman_p_value}")
  print(f"Pearson correlation: {pearson_corr}, p-value: {pearson_p_value}")

In [ ]:
df['category_code_biotech'] = df['category_code'].apply(lambda x: 1 if x == 'biotech' else 0) #only 34 samples with 1
print_correlations_Spearman_and_Pearson(df['category_code_biotech'], df['status_code'])

In [ ]:
print_correlations_Spearman_and_Pearson(df['relationships'], df['status_code'])

In [ ]:
print_correlations_Spearman_and_Pearson(df['avg_participants'], df['status_code'])

In [ ]:
print_correlations_Spearman_and_Pearson(df['has_roundC'], df['status_code'])

In [ ]:
print_correlations_Spearman_and_Pearson(df['has_roundD'], df['status_code'])

In [ ]:
# nasdaq is a better prediction feature then inflation
print('inflation_at_year_0')
print_correlations_Spearman_and_Pearson(df['inflation_at_year_0'], df['status_code'])
print('')
print('nasdaq_annual_changes_at_year_0')
print_correlations_Spearman_and_Pearson(df['nasdaq_annual_changes_at_year_0'], df['status_code'])

In [ ]:
#print_correlations_Spearman_and_Pearson(df['gdp_growth_at_year_0'], df['status_code'])